In [2]:
import Ipynb_importer
from a_basic_quant import *
from b_model import *
# from ipywidgets import IntProgress

importing Jupyter notebook from a_basic_quant.ipynb
importing Jupyter notebook from b_model.ipynb


In [1]:
import torch
import torch.optim as optim
from torchvision import datasets, transforms
import os
import sys

if sys.platform == 'linux':
    dataset =  '/home/xia/Dataset'
else:
    dataset = r'C:\Users\xia\Documents\datasets'
dataset

'C:\\Users\\xia\\Documents\\datasets'

### 1、定义训练

In [4]:
def train(model, device, train_loader, criterion, optimizer, epoch):
    model.train()
    train_loss = 0
    correct = 0
    for batch_idx, (datas, targets) in enumerate(train_loader):
        datas, targets = datas.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(datas)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        train_loss += loss
        pred = outputs.argmax(dim=1, keepdim=True)
        correct += pred.eq(targets.view_as(pred)).sum().item()
        
    train_loss /= len(train_loader.dataset)
        
    print('Train Epoch:{} \t  Average loss: {:.4f}, Accuracy: {}/{}'.format(
        epoch, train_loss, correct , len(train_loader.dataset)
    ))

### 2、定义测试

In [5]:
def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    for batch_idx, (datas, targets) in enumerate(test_loader):
        datas, targets = datas.to(device), targets.to(device)
        outputs = model(datas)
        loss = criterion(outputs, targets)
        
        test_loss += loss.item()
        pred = outputs.argmax(dim=1, keepdim=True)
        correct += pred.eq(targets.view_as(pred)).sum().item()
        
    test_loss /= len(test_loader.dataset)
    
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{}'.format(
        test_loss,  correct ,  len(test_loader.dataset)
    ))

### 3、定义数据集和loader

In [6]:
def dataset_loader(batch_size, test_batch_size):
    train_transform = transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])
    test_transform = transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])
    
    train_dataset = datasets.MNIST(dataset, train=True, download=True,
                                  transform=train_transform)
    test_dataset = datasets.MNIST(dataset, train=False, download=True,
                                 transform=test_transform)  
    
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                              shuffle=True, num_workers=1)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size,
                                             shuffle=False, num_workers=1)
    
    return train_loader, test_loader

In [7]:
def main():
    batch_size = 256
    test_batch_size = 64
    seed = 1
    epochs = 25
    lr = 0.01
    momentum = 0.7
    save_model = True
    using_bn = True
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    if device == 'cuda':
        torch.cuda.manual_seed(seed)
    else:
        torch.manual_seed(seed)
    
    # 制作 loader
    train_loader, test_loader = dataset_loader(batch_size, test_batch_size)
    
    if using_bn:
        model = NetBN().to(device)
    else:
        model = Net().to(device)
        
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
    criterion = torch.nn.CrossEntropyLoss()
    
    for epoch in range(epochs):
        if epoch == 18:     lr = 0.001
        train(model, device, train_loader, criterion, optimizer, epoch)
        test(model, device, test_loader, criterion)
    
    if save_model:
        if not os.path.exists('ckpt'):
            os.makedirs('ckpt')
        if using_bn:
            torch.save(model.state_dict(), 'ckpt/mnist_cnnbn.pt')
        else:
            torch.save(model.state_dict(), 'ckpt/mnist_cnn.pt')

In [ ]:
if __name__ == "__main__":
    main()

In [22]:
# train_loader, test_loader = dataset_loader(64, 64)
# for index,item in enumerate(train_loader):
#     datas,_ = item
#     print(datas.shape)
#     break

torch.Size([64, 1, 28, 28])
